In [ ]:
source("utils/plot.R")

In [ ]:
read_data <- function(filename) {
    df <- read_tsv(filename, col_types=cols(), skip=2) %>%
        filter(percent_rank(Position) > 0.01, percent_rank(Position) < 0.99)
    return(df)
}

In [ ]:
known_sweeps <- tibble(
    locus=c("ace", "chkov", "cyp"),
    pos=c(9069408, 21150000, 8072884),
    chrom=c("3R", "3R", "2R")
)

In [ ]:
known_sweeps

In [ ]:
find_proximity <- function(df, bp_distance, known_sweeps) {
    result <- c()
    for (row in 1:nrow(df)) {
        x <- df$Position[row]
        res <- NA
        for (ix in 1:nrow(known_sweeps)) {
            right_chrom <- known_sweeps$chrom[ix]
            if ((df$chrom[row] == right_chrom) && (between(x, known_sweeps$pos[ix] - bp_distance/2, known_sweeps$pos[ix] + bp_distance/2))) {
                res <- known_sweeps$locus[ix]
            }
        }
        result <- c(result, res)
    }
    return(result)
}

In [ ]:
input_dfs <- lapply(snakemake@input, read_data)
names(input_dfs) <- snakemake@input

df <- bind_rows(input_dfs, .id="path") %>%
    separate(path, into=c(NA, NA, NA, "filename"), sep="/") %>%
    separate(filename, into=c(NA, "chrom", NA), sep=c(4, 6))

df <- mutate(df, proximity=find_proximity(df, 1e6, known_sweeps))

In [ ]:
best_likelihood_near_loci <- df %>%
    filter(!is.na(proximity)) %>%
    group_by(proximity) %>%
    top_n(1, Likelihood) %>%
    mutate(AlphaLabel=paste0("alpha = ", signif(Alpha, 2)))

In [ ]:
fig <- ggplot(df) +
    geom_point(aes(x=Position/1e6, y=Likelihood), size=0.5) +
    geom_vline(
        data=known_sweeps,
        aes(xintercept=pos/1e6),
        linetype="dashed",
        colour="darkgrey"
    ) +
    geom_point(
        data=best_likelihood_near_loci,
        aes(x=Position/1e6, y=Likelihood),
        colour="darkred",
        size=1
    ) +
    geom_label(
        data=best_likelihood_near_loci,
        aes(x=Position/1e6, y=Likelihood, label=AlphaLabel),
        hjust=0,
        nudge_x=1,
        size=1.5
    ) +
    facet_wrap(vars(chrom), scales="free_x") +
    labs(
        x="Position (Mb)",
        y="Likelihood"
    ) +
    sweeps_theme +
    theme(
        axis.text.y=element_blank()
    )

sweeps_save(snakemake@output[[1]], fig)